**Aluno**: Lucas Peres Gaspar

**Matrícula**: 409504

**Nível**: Mestrando

**Programa**: Mestrado e Doutorado em Ciência da Computação

---

O objetivo deste trabalho é clusterizar um conjunto de dados utilizando o algoritmo *K-Médias* e analisar o comportamento do resultado de acordo com o número de clusters. O código foi desenvolvido em Python 3 utilizando as bibliotecas Numpy, Pandas e o Scikit-Learn(a fins de otimização), bem como o ambiente de programação Jupyter Notebook. Este trabalho encontra-se no [GitHub](https://github.com/lucaspg96/pattern-recognition/tree/work3/work3), assim como os códigos-fonte.

Primeiramente, devemos importar as bibliotecas que serão utilizadas durante as análises.

In [1]:
import numpy as np
import pandas as pd

from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

Utilizamos o Pandas para visualizar os dados de maneira tabular, a fim de identificar como os dados estão organizados.

In [2]:
df = pd.read_csv("data.tsv",delimiter="\t", header=None)
df.head()

,0,1,2,3,4,5
0,100.0,2.000000,477.750000,3.464102,3.464102,10
1,50.0,4.250000,975.496844,1.477098,1.596796,10001
2,100.0,3.214286,3747.113679,1.022928,0.984565,15577
3,100.0,3.153846,4906.374000,0.831080,0.858073,22042
4,100.0,4.000000,132.100000,3.464102,3.464102,22227


Para este trabalho, precisamos, inicialmente, normalizar os dados. Optamos por utilizar normalização de ordem infinita, ou seja, normalizamos pelo valor máximo.

In [3]:
data = df.values
data = data / data.max(axis=0)
data.shape

(1701, 6)

# Métricas
---

Para podermos avaliar os resultados da clusterização, precisaremos implementar algumas métricas para os clusteres gerados. Contaremos, neste trabalho, com 4 métricas:

* Índice **Dunn**, que analiza a menor distância entre os clusters e a maior distância dentro de um cluster;
* Índice **Davies-Bouldin**, que analiza a dispersão intra e inter-grupos;
* Índice **Calinski-Harabasz**, que analiza as matrizes de dispersão intra e inter-grupos;
* **Silhueta**, que representa a distância média intra-grupos.

In [4]:
def min_dist(clusters):
    m_d = 10000000
    for c1 in clusters:
        for c2 in clusters:
            if not c2 == c1:
                for x1 in clusters[c1]:
                    for x2 in clusters[c2]:
                        d = np.linalg.norm(x1-x2)
                        if d < m_d:
                            m_d = d
                    
    return m_d

def max_dist(clusters):
    m_d = 0
    for c1 in clusters:
        for x1 in clusters[c1]:
            for x2 in clusters[c1]:
                d = np.linalg.norm(x1-x2)
                if d > m_d:
                    m_d = d
                    
    return m_d

def dunn(clusters):
    return min_dist(clusters)/max_dist(clusters)

In [5]:
import math

def Siq(w,X,q=2):
    return math.pow(np.mean([np.linalg.norm(x-w,q) \
                             for x in X]),1/q)

def Dijt(wi,wj,t=2):
    return np.linalg.norm(wi-wj,t)

def Riqt(clusters,centroids,i,q=2,t=2):
    return max(\
           [(Siq(centroids[i],clusters[i],q) +\
            Siq(centroids[j],clusters[j],q))\
            /Dijt(centroids[i],centroids[j],t) for \
              j in clusters.keys()-[i]])

def davies_bouldin(clusters,centroids):
    rs = [\
            Riqt(clusters,centroids,i) for \
            i in clusters]
    
    return np.mean(rs)

In [6]:
def Bk(centroids,nis,m):
    n = m.shape[0]
    bk = np.zeros((n,n))
    for c,ni in zip(centroids,nis):
        z = (c-m).reshape((1,n))
        bk += ni* (z.T * z)
        
    return bk

def Wk(clusters, centroids):
    n = centroids.shape[1]
    wk = np.zeros((n,n))
    
    for i in clusters:
        for x in clusters[i]:
            z = (x-centroids[i]).reshape(1,n)
            wk += z.T * z
            
    return wk

def calinski_harabasz(clusters, centroids,m):
    k = len(centroids)
    nis = [clusters[i].shape[0] for i in range(k)]

    bk = Bk(centroids,nis,m)
    
    wk = Wk(clusters, centroids)
    
    n = sum(nis)
    
    return (np.trace(bk)/(k-1))/(np.trace(wk)/(n-k))

# Clusterização
---

Agora que temos as métricas, vamos analizar as clusterizações, variando o número de clusters de 2 à 10: 

In [7]:
df = []
m = np.mean(data,axis=0)
models = {}
for k in range(2,11):
    model = KMeans(n_clusters=k)
    clusters = {i: [] for i in range(k)}
    
    y = model.fit_predict(data)
    models[k] = model
    
    for c,x in zip(y,data):
        clusters[c].append(x)
        
    for c in clusters:
        clusters[c] = np.array(clusters[c])
    
    df.append({
        "1-K":k,
        "2-Dunn": dunn(clusters),
        "3-DB": davies_bouldin(clusters,model.cluster_centers_),
        "4-CH":calinski_harabasz(clusters,model.cluster_centers_,m),
        "5-SL":silhouette_score(data,y)
    })
        
df = pd.DataFrame(df)
df

,1-K,2-Dunn,3-DB,4-CH,5-SL
0,2,0.380200,1.359383,1917.547799,0.541584
1,3,0.016003,1.065287,2150.362182,0.587314
2,4,0.019874,1.079467,4325.884105,0.657097
3,5,0.013747,1.363054,4720.415403,0.653328
4,6,0.014434,1.509995,6460.105474,0.652840
5,7,0.009770,1.743611,6656.942135,0.640673
6,8,0.009770,1.883052,6874.657130,0.624277
7,9,0.015238,1.812652,7002.028051,0.629830
8,10,0.018723,2.061325,6942.863495,0.614686


Analisando as métricas, temos que:

* **Dunn** iddentifica como partição válida ótima K=2, seguido, bem distante, de K=4;
* **DB** iddentifica como partição válida ótima K=3, seguido de K=4;
* **CH** iddentifica como partição válida ótima K=7, seguida de K=8;
* **SL** iddentifica como partição válida ótima K=4, seguido de K=5.

Podemos notar uma divergência entre as partições ótimas de acordo com as métricas. O fato do **Dunn** ter dado valores tão discrepantes entre K=2 e os demais deve-se ao fato de ele ser muito sensível à outliers: mesmo que uma partição esteja bem distribuída, caso ela tenha um ponto mais distânte dos outros, isso pode diminuir o valor do índice.

Como **SL** e **DB** analizam as dispersões intra e inter-grupos, é natural que haja uma intersecção entre seus valores. Quanto ao **CH**, ele é sensível à variância dos dados, dando melhores valores para dados menos dispersos.

Como K=4 foi o que mais apareceu dentre os melhores das métricas, vamos fazer a análise de seu resultado. Para cada partição, fazemos uma análise estatística dos atributos dos dados. Para isso, utilizamos a função *describe* do pandas.

In [9]:
k = 4
model = models[k]
print("{} clusters:".format(k))

y = model.predict(data)

for c in range(k):
    d = data[y==c]
    stats_df = pd.DataFrame(d)
    print("\nC{} - {}\n".format(c, model.cluster_centers_[c]))
    print(stats_df.describe())

4 clusters:

C0 - [ 0.99002176  0.05644289  0.00209818  0.88502816  0.89588167  0.02365278]

                0           1           2           3           4           5
count  383.000000  383.000000  383.000000  383.000000  383.000000  383.000000
mean     0.990022    0.056443    0.002098    0.885028    0.895882    0.023653
std      0.084198    0.055188    0.004543    0.154738    0.143877    0.027805
min      0.000000    0.013761    0.000326    0.564076    0.674200    0.000001
25%      1.000000    0.027523    0.000503    0.674200    0.717741    0.008340
50%      1.000000    0.041284    0.000899    1.000000    1.000000    0.013056
75%      1.000000    0.061927    0.001719    1.000000    1.000000    0.013840
max      1.000000    0.811927    0.057802    1.000000    1.000000    0.097668

C1 - [ 0.99386413  0.0586311   0.0126536   0.43470973  0.45303269  0.93951992]

                0           1           2           3           4           5
count  547.000000  547.000000  547.000000  547

Analisando as partições, podemos observar algumas características:

* Os elementos estão bem distribuídos entre as partições, sendo a partição 1 a com mais (547) e a 3 com menos (343);
* O atributo 0 é o que sofre menos modificação, sendo bem semelhante entre todas as partições;
* Os atributos 3 e 4 são os mais variantes
* O uso de 4 partições apresentou uma boa separação entre os dados.

Mesmo tendo as métricas de dispersão os dados e as métricas dos particionamentos, o resultado não pode ser conclusivo. Tem-se de estudar mais sobre os significado de cada atributo, a fim de entender melhor o que está sendo representado nas partições.